In [1]:
ls

 Volume in drive C has no label.
 Volume Serial Number is E25B-F2F8

 Directory of C:\Users\LENOVO\servermcp

17-08-2025  09:56    <DIR>          .
17-08-2025  09:56    <DIR>          ..
17-08-2025  09:55               109 .gitignore
17-08-2025  09:54    <DIR>          .ipynb_checkpoints
17-08-2025  09:55                 5 .python-version
17-08-2025  09:56    <DIR>          .venv
17-08-2025  09:55                87 main.py
17-08-2025  09:56               195 pyproject.toml
17-08-2025  09:55                 0 README.md
17-08-2025  09:54               337 server.ipynb
17-08-2025  09:56            93,235 uv.lock
               7 File(s)         93,968 bytes
               4 Dir(s)  12,254,175,232 bytes free


In [3]:
%%writefile C:\Users\LENOVO\servermcp/research_server.py

import arxiv
import json
import os
import re
from typing import List
from mcp.server.fastmcp import FastMCP


PAPER_DIR = "papers"

# Initialize FastMCP server
mcp = FastMCP("research")

@mcp.tool()
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # DEFENSIVE: Clean and sanitize the topic input
    if not isinstance(topic, str):
        topic = str(topic)
    
    # Remove control characters and clean whitespace
    topic = ''.join(char for char in topic if ord(char) >= 32 and char not in '<>:"/\\|?*')
    topic = topic.strip()
    
    # Validate that we have a real topic
    if not topic:
        raise ValueError("Topic cannot be empty")
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic - SANITIZED
    safe_topic = re.sub(r'[<>:"/\\|?*\x00-\x1F]', '_', topic.lower())
    safe_topic = re.sub(r'_+', '_', safe_topic)  # Replace multiple underscores
    safe_topic = safe_topic.strip('_')  # Remove leading/trailing underscores
    
    # Handle empty topic name edge case
    if not safe_topic:
        safe_topic = "untitled"
    
    path = os.path.join(PAPER_DIR, safe_topic)
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

@mcp.tool()
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
    
    # Validate paper_id
    if not paper_id or not isinstance(paper_id, str):
        return "Invalid paper ID provided"
    
    # Clean paper_id
    paper_id = paper_id.strip()
    
    # Check if papers directory exists
    if not os.path.exists(PAPER_DIR):
        return f"No papers directory found. Please run a search first."
    
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."



if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting C:\Users\LENOVO\servermcp/research_server.py
